1. Access the first page of the product using requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

### Extraction start here 

In [ ]:
with open("./cookie.json", 'r') as f:
    headers = json.load(f)

In [ ]:
url = 'https://www.ceneo.pl/98925064#tab=reviews'
all_opinions = []
while url:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        opinions = soup.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion= {}
            single_opinion['opinion_id'] = opinion['data-entry-id']
            single_opinion['author'] = opinion.select_one('span.user-post__author-name').text.strip()
            try:
                single_opinion['recommend'] = opinion.select_one('span.user-post__author-recomendation > em.recommended').text.strip()
            except AttributeError:
                single_opinion['recommend'] = None
            single_opinion['stars'] = opinion.select_one('span.user-post__score-count').text.strip()
            single_opinion['content'] = opinion.select_one('div.user-post__text').text.strip()
            single_opinion['pros'] = [tag.text for tag in opinion.select('div.review-feature__item--positive')]
            single_opinion['cons'] = [tag.text for tag in opinion.select('div.review-feature__item--negative')]
            single_opinion['helpful'] = opinion.select_one('button.vote-yes')["data-total-vote"].strip()
            single_opinion['unhelpful'] = opinion.select_one('button.vote-no')["data-total-vote"].strip()
            single_opinion['published'] = opinion.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip()
            try:
                single_opinion['purchased'] = opinion.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip()
            except TypeError:
                single_opinion['purchased'] = None
            all_opinions.append(single_opinion)
    try:
        url ='https://www.ceneo.pl' + soup.select_one('a.pagination__next')['href']
    except TypeError:
        url = None
print(all_opinions)
